# INIT

START

In [ ]:
import sys
import pprint
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

import ipytest
ipytest.autoconfig()
from time import sleep

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string, base64_to_hex, nominated_amount
from utils.logger import get_logger

from contracts.farm_contract import FarmContract
from contracts.metastaking_contract import MetaStakingContract
from contracts.staking_contract import StakingContract
from utils.contract_retrievers import retrieve_farm_by_address

from multiversx_sdk import ProxyNetworkProvider, ApiNetworkProvider

logger = get_logger("manual_interactor")

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

In [ ]:
metastaking_contract: MetaStakingContract = context.deploy_structure.get_deployed_contract_by_index(config.METASTAKINGS_V2, 0)
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 0)
staking_contract: StakingContract = context.deploy_structure.get_deployed_contract_by_index(config.STAKINGS_V2, 0)

print(f"Using {metastaking_contract.address} : {metastaking_contract.metastake_token}")
pprint.pprint(metastaking_contract.get_config_dict())
pprint.pprint(staking_contract.get_config_dict())
pprint.pprint(farm_contract.get_config_dict())

CHAIN CONFIG SETUP

In [268]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

USERS = ["erd1n0q029ywlacwgp22llvcr9zyuvx3ntqhmwf7ztx364f5wrjauhqsxdgryt", # user with farm & farming positions
         "erd14kfw9hwuaz2e5sfgcsn9rxrc2k24fnea3zgn0nnh9g4374ssggssf49gtd", # user with only farm position
         "erd1e6whhkrm7254xw0y2vr4hv7vft2a84gpdmf27v8yyttpzpqydavs7v8fav", # user with only farm position
         "erd15gzp9k56cnn8qtfxwlghcxgs74v8jmfk4crex9alygxldmpg9f5s3fd4pl" # user with old farm position
        ]
def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    

def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", state)


def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})


def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

CHAIN SIMULATOR STACK

In [269]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH)

Farm upgrade

In [270]:
def farm_upgrade():
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/farm-with-locked-rewards.wasm", 
                                            [], True)

    advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).contract_code_hash.hex()
    assert code_hash == "4ca038c097412353ec6cf6039f2a216aa70725d80d2a647b7948bc466d0d4444"

Metastaking upgrade

In [271]:
def metastaking_upgrade():
    tx_hash = metastaking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                                    "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/farm-staking-proxy.wasm",
                                                    [], True)

    advance_blocks(2)
    
    code_hash = context.network_provider.proxy.get_account(WrapperAddress(metastaking_contract.address)).contract_code_hash.hex()
    assert code_hash == "8a605fa8539f193a49d267ce73f9010904796d6b930ea7e4e09f9b4d9da1af9f"


Staking upgrade

In [272]:
def staking_upgrade():
    tx_hash = staking_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                                "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/farm-staking.wasm",
                                                [], True)

    advance_blocks(1)
    tx_hash = staking_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).contract_code_hash.hex()
    assert code_hash == "5ae911b77e1fdb0369d82cbfbe1db463a18b9cbcd1cb68f5514cddb3d85df8aa"

Deploy permissions hub

In [273]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    permissions_hub_contract = PermissionsHubContract("")
    _, address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/permissions-hub.wasm",
                                            [])
    permissions_hub_contract.address = address
    return permissions_hub_contract

Dummy proxy

In [274]:
from contracts.dummy_proxy_contract import DummyProxyContract

def deploy_dummy_proxy_contract():
    dummy_proxy_contract = DummyProxyContract("")
    _, address = dummy_proxy_contract.contract_deploy(context.deployer_account, context.network_provider.proxy,
                                         "https://github.com/ovidiuolteanu/mx-sc-dummy-proxy/releases/download/v2.1/dummy-proxy.wasm",
                                         [])
    dummy_proxy_contract.address = address
    return dummy_proxy_contract

In [275]:
from utils.utils_chain import get_all_token_nonces_details_for_account

def get_position_for_account(user_address: str):
    metastake_tk_balance, metastake_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > metastake_tk_balance:
            metastake_tk_balance = int(token['balance'])
            metastake_tk_nonce = token['nonce']
            break

    if not metastake_tk_nonce:
        raise Exception("Not enough metastake token balance")
    
    return metastake_tk_nonce, metastake_tk_balance

def get_farming_position_for_account(user_address: str):
    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    return farm_tk_nonce, farm_tk_balance

In [276]:
from utils.utils_tx import ESDTToken, multi_esdt_transfer

def send_tokens_to_dummy(user: Account, dummy_contract: DummyProxyContract):
    farm_tk_nonce, farm_tk_balance = get_position_for_account(user)
    print(f"Sending {farm_tk_balance} {metastaking_contract.metastake_token}-{farm_tk_nonce} to dummy contract")
    multi_esdt_transfer(context.network_provider.proxy, 20000000, user, dummy_contract.address, [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)])

Claim

In [277]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def claim_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough metastake token balance")
    
    tokens = [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)]

    tx_hash = metastaking_contract.claim_rewards_metastaking(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Claim on behalf

In [278]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def claim_on_behalf_from_user(claim_account: Account):
    claim_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, claim_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    tokens = [ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance)]

    tx_hash = metastaking_contract.claim_rewards_on_behalf_metastaking(context.network_provider.proxy, claim_account, [tokens])
    return tx_hash

Enter farm consolidated

In [308]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            
            tokens.append(ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance))

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    tx_hash = metastaking_contract.enter_metastake(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Enter farm no consolidation

In [280]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_no_consolidation_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]
    tx_hash = metastaking_contract.enter_metastake(context.network_provider.proxy, user_account, [tokens])
    return tx_hash

Enter farm on behalf

In [309]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_on_behalf_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            
            tokens.append(ESDTToken(metastaking_contract.metastake_token, farm_tk_nonce, farm_tk_balance))

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    tx_hash = metastaking_contract.enter_metastake_on_behalf(context.network_provider.proxy, caller_account, [tokens, user_account.address.bech32()])
    return tx_hash

Enter farm on behalf no consolidation

In [282]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account

def enter_farm_on_behalf_no_consolidation_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance, farming_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            farming_tk_nonce = token['nonce']
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    tokens = [ESDTToken(metastaking_contract.farm_token, farming_tk_nonce, farming_tk_balance)]

    tx_hash = metastaking_contract.enter_metastake_on_behalf(context.network_provider.proxy, caller_account, [tokens, user_account.address.bech32()])
    return tx_hash

Utilities

In [283]:
from utils.decoding_structures import METASTAKE_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

def user_farm_token_stats(user: Account):
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.farm_token, user.address.bech32(), context.network_provider.proxy)
    print(f'Account: {user.address.bech32()}')
    print(f'Looking for {metastaking_contract.farm_token} and {metastaking_contract.metastake_token} tokens')
    print(f'Farming Tokens in account:')
    for token in tokens_in_account:
        print(f'\t{token}')
    tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)
    print(f'Farm Tokens in account:')
    all_decoded_attributes = []
    for token in tokens_in_account:
        print(f'\t{token}')
        decoded_attributes = metastaking_contract.get_all_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), token['nonce'])
        print(f'\t\t{decoded_attributes}')
        all_decoded_attributes.append(decoded_attributes)
        
    return all_decoded_attributes

In [ ]:
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum, auto
from collections import defaultdict

class DictType(Enum):
    USER_FARM_STATS_1 = auto()
    USER_STAKING_STATS_1 = auto()
    USER_2_FARM_STATS_1 = auto()
    USER_2_STAKING_STATS_1 = auto()
    PROXY_FARM_STATS_1 = auto()
    PROXY_STAKING_STATS_1 = auto()
    FARM_CONTRACT_STATS_1 = auto()
    STAKING_CONTRACT_STATS_1 = auto()
    TOKEN_ATTRS_1 = auto()
    USER_FARM_STATS_2 = auto()
    USER_STAKING_STATS_2 = auto()
    USER_2_FARM_STATS_2 = auto()
    USER_2_STAKING_STATS_2 = auto()
    PROXY_FARM_STATS_2 = auto()
    PROXY_STAKING_STATS_2 = auto()
    FARM_CONTRACT_STATS_2 = auto()
    STAKING_CONTRACT_STATS_2 = auto()
    TOKEN_ATTRS_2 = auto()
    OP1 = auto()
    OP2 = auto()
    OP3 = auto()
    OP4 = auto()

@dataclass
class DictComparison:
    before: Dict[str, Any]
    after: Dict[str, Any]
    description: str
    phase: str

class DictCollector:
    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all collections for a new scenario"""
        self.before_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.after_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.current_phase = "before"  # "before" or "after"
    
    def set_phase(self, phase: str):
        """Set the current collection phase"""
        if phase not in ["before", "after"]:
            raise ValueError("Phase must be 'before' or 'after'")
        self.current_phase = phase

    def add(self, dict_type: DictType, dict_data: Dict[str, Any], description: str = ""):
        """Add a dictionary to the current phase collection"""
        if self.current_phase == "before":
            self.before_upgrade[dict_type].append((dict_data, description))
        else:
            self.after_upgrade[dict_type].append((dict_data, description))

    def _compare_dicts(self, dict1: Any, dict2: Any) -> Tuple[bool, Optional[str]]:
        """
        Compare two objects that can be either dictionaries or lists of dictionaries.
        Returns (is_equal, difference_description)
        """
        # Handle lists of dictionaries
        if isinstance(dict1, list) and isinstance(dict2, list):
            if len(dict1) != len(dict2):
                return False, f"Different list lengths: {len(dict1)} != {len(dict2)}"
            
            for i, (item1, item2) in enumerate(zip(dict1, dict2)):
                if isinstance(item1, (dict, list)) and isinstance(item2, (dict, list)):
                    is_equal, diff = self._compare_dicts(item1, item2)
                    if not is_equal:
                        return False, f"List item {i} difference: {diff}"
                elif item1 != item2:
                    return False, f"List item {i} mismatch: {item1} != {item2}"
            return True, None

        # Handle dictionaries
        if isinstance(dict1, dict) and isinstance(dict2, dict):
            if dict1.keys() != dict2.keys():
                missing_keys = set(dict1.keys()) - set(dict2.keys())
                extra_keys = set(dict2.keys()) - set(dict1.keys())
                return False, f"Different keys. Missing: {missing_keys}, Extra: {extra_keys}"

            for key in dict1:
                if isinstance(dict1[key], (dict, list)) and isinstance(dict2[key], (dict, list)):
                    is_equal, diff = self._compare_dicts(dict1[key], dict2[key])
                    if not is_equal:
                        return False, f"Nested difference at key '{key}': {diff}"
                elif dict1[key] != dict2[key]:
                    return False, f"Value mismatch for key '{key}': {dict1[key]} != {dict2[key]}"
            return True, None

        # Handle case where types don't match
        if type(dict1) != type(dict2):
            return False, f"Type mismatch: {type(dict1)} != {type(dict2)}"

        # Handle other cases
        return dict1 == dict2, f"Value mismatch: {dict1} != {dict2}" if dict1 != dict2 else None

    def compare_all(self) -> List[str]:
        """
        Compare all collected dictionary pairs and return a list of differences found.
        """
        differences = []

        for dict_type in DictType:
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            # Check if we have matching pairs
            if len(before_list) != len(after_list):
                differences.append(
                    f"{dict_type.name}: Mismatched number of collections - "
                    f"Before: {len(before_list)}, After: {len(after_list)}"
                )
                continue

            # Compare each pair
            for i, ((before_dict, before_desc), (after_dict, after_desc)) in enumerate(zip(before_list, after_list)):
                is_equal, diff = self._compare_dicts(before_dict, after_dict)
                if not is_equal:
                    diff_msg = f"{dict_type.name} comparison failed"
                    if before_desc or after_desc:
                        diff_msg += f" (Before: {before_desc}, After: {after_desc})"
                    diff_msg += f": {diff}"
                    differences.append(diff_msg)

        return differences
    
    def print_collections(self):
        """
        Print a formatted view of all collections before and after upgrade.
        """
        print("\nCollections Summary:")
        print("=" * 80)

        for dict_type in DictType:            
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            if not before_list and not after_list:
                continue

            print(f"\n{dict_type.name}:")
            print("-" * 40)

            print("\nBefore upgrade:")
            if not before_list:
                print("  No collections")
            for i, (data, desc) in enumerate(before_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\nAfter upgrade:")
            if not after_list:
                print("  No collections")
            for i, (data, desc) in enumerate(after_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\n" + "=" * 80)

# Example usage:
"""
collector = DictCollector()

# Before upgrade
collector.set_phase("before")
collector.add(DictType.USER_STATS, u1, "Initial user stats")
collector.add(DictType.CONTRACT_STATS, c1, "Initial contract stats")
collector.add(DictType.TOKEN_ATTRS, tk_attrs_1, "Initial token attributes")
collector.add(DictType.CLAIM_OPS, claim_ops_1, "Initial claim operations")

# After upgrade
collector.set_phase("after")
collector.add(DictType.USER_STATS, u3, "Post-upgrade user stats")
collector.add(DictType.CONTRACT_STATS, c3, "Post-upgrade contract stats")
collector.add(DictType.TOKEN_ATTRS, tk_attrs_2, "Post-upgrade token attributes")
collector.add(DictType.CLAIM_OPS, claim_ops_3, "Post-upgrade claim operations")

# Compare results
differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")

# Alternatively, at the end of your test
differences = collector.compare_all()
assert not differences, "\n".join(differences)

# Reset for next scenario
collector.reset()
"""

In [304]:
def collect_initial_test_data(collector: DictCollector, user: Account, farm_contract: FarmContract, staking_contract: StakingContract):
    u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    c1 = farm_contract.get_all_stats(context.network_provider.proxy)
    su1 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    s1 = staking_contract.get_all_stats(context.network_provider.proxy)
    tk_attrs_1 = user_farm_token_stats(user)

    collector.add(DictType.USER_FARM_STATS_1, u1, f"Initial farm user stats")
    collector.add(DictType.FARM_CONTRACT_STATS_1, c1, f"Initial farm contract stats")
    collector.add(DictType.USER_STAKING_STATS_1, su1, f"Initial staking user stats")
    collector.add(DictType.STAKING_CONTRACT_STATS_1, s1, f"Initial staking contract stats")
    collector.add(DictType.TOKEN_ATTRS_1, tk_attrs_1, f"Initial token attributes")

def collect_ending_test_data(collector: DictCollector, user: Account, farm_contract: FarmContract, staking_contract: StakingContract,
                             dummy_proxy_contract: DummyProxyContract = None):
    u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    c2 = farm_contract.get_all_stats(context.network_provider.proxy)
    su2 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
    s2 = staking_contract.get_all_stats(context.network_provider.proxy)
    tk_attrs_2 = user_farm_token_stats(user)
    if dummy_proxy_contract:
        pf1 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
        ps1 = staking_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
        tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))


    collector.add(DictType.USER_FARM_STATS_2, u2, f"Ending farm user stats")
    collector.add(DictType.FARM_CONTRACT_STATS_2, c2, f"Ending farm contract stats")
    collector.add(DictType.USER_STAKING_STATS_2, su2, f"Ending staking user stats")
    collector.add(DictType.STAKING_CONTRACT_STATS_2, s2, f"Ending staking contract stats")
    collector.add(DictType.TOKEN_ATTRS_2, tk_attrs_2, f"Ending token attributes")
    if dummy_proxy_contract:
        collector.add(DictType.PROXY_FARM_STATS_2, pf1, f"Ending proxy farm stats")
        collector.add(DictType.PROXY_STAKING_STATS_2, ps1, f"Ending proxy staking stats")

def report_test_data(collector: DictCollector):
    collector.print_collections()
    
    differences = collector.compare_all()
    if differences:
        print("Found differences:")
        for diff in differences:
            print(f"- {diff}")
    else:
        print("All comparisons passed!")
        
    assert not differences, "\n".join(differences)

In [301]:
def init_pre_update_test(initial_blocks: int) -> Tuple[Any, List[Account], DictCollector, int]:
    chain_sim = start_chain_sim_stack()

    setup_chain_sim()
    users = users_init()

    advance_blocks(initial_blocks)

    collector = DictCollector()
    collector.set_phase("before")

    return chain_sim, users, collector

def init_post_upgrade_test(initial_blocks: int, chain_sim: Any, collector: DictCollector):
    input("Restart Chain Simulator then press Enter to continue...")
    
    stop_chain_sim_stack(chain_sim)
    chain_sim = start_chain_sim_stack()

    setup_chain_sim()
    users = users_init()
    advance_blocks(1)
    consumed_blocks = 1

    farm_upgrade()  # eats 2 blocks
    consumed_blocks += 2
    staking_upgrade()  # eats 2 blocks
    consumed_blocks += 2
    metastaking_upgrade()  # eats 2 blocks
    consumed_blocks += 2

    permissions_hub_contract = deploy_permissions_hub() # eats 1 block
    consumed_blocks += 1

    farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
    advance_blocks(1)
    consumed_blocks += 1

    staking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
    advance_blocks(1)
    consumed_blocks += 1

    metastaking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
    advance_blocks(1)
    consumed_blocks += 1

    dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
    consumed_blocks += 1


    permissions_hub_contract.add_to_whitelist(users[0], context.network_provider.proxy, [dummy_proxy_contract.address])
    permissions_hub_contract.add_to_whitelist(users[1], context.network_provider.proxy, [dummy_proxy_contract.address])
    permissions_hub_contract.add_to_whitelist(users[2], context.network_provider.proxy, [dummy_proxy_contract.address])
    advance_blocks(1)
    consumed_blocks += 1

    if initial_blocks < consumed_blocks:
        raise Exception(f"Initial blocks {initial_blocks} is less than consumed blocks {consumed_blocks}. Skewed results will occur.")
    
    block_diff = initial_blocks - consumed_blocks
    advance_blocks(block_diff)

    collector.set_phase("after")

    return chain_sim, users, permissions_hub_contract, dummy_proxy_contract

def close_test(chain_sim: Any, collector: DictCollector):
    report_test_data(collector)
    
    print("Chain simulator need to be stopped manually...")
    # stop_chain_sim_stack(chain_sim)

In [287]:
def dict_compare(d1, d2):
    print(d1)
    print(d2)
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

def check_equal_dicts(dict1, dict2):
    """
    Compare two dictionaries, including nested dictionaries.
    
    Args:
    dict1 (dict): First dictionary to compare.
    dict2 (dict): Second dictionary to compare.
    
    Returns:
    bool: True if dictionaries are equal, False otherwise.
    """
    if dict1.keys() != dict2.keys():
        return False
    
    for key in dict1:
        if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            if not check_equal_dicts(dict1[key], dict2[key]):
                return False
        elif dict1[key] != dict2[key]:
            return False
    
    return True

# SCENARIOS

Chain sim control

In [ ]:
chain_sim_stack = start_chain_sim_stack()

In [ ]:
stop_chain_sim_stack(chain_sim_stack)

Init

In [ ]:
setup_chain_sim()
users = users_init()
advance_blocks(1)

In [ ]:
user_farm_token_stats(users[1])

In [ ]:
from tools import chain_simulator_connector
user = users_init()[1]
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
metastaking_token_attributes = metastaking_contract.get_decoded_metastake_token_attributes_from_proxy(context.network_provider.proxy, user.address.bech32(), user_tk_nonce)
farm_token = ESDTToken(metastaking_contract.farm_token, metastaking_token_attributes['lp_farm_token_nonce'], 0)
staking_token = ESDTToken(metastaking_contract.stake_token, metastaking_token_attributes['staking_farm_token_nonce'], 0)

# chain_simulator_connector.main(["--gateway", context.network_provider.proxy.url, "--token", farm_token.get_full_token_name()])
# chain_simulator_connector.main(["--gateway", context.network_provider.proxy.url, "--token", staking_token.get_full_token_name()])
print(farm_token.get_full_token_name())
print(staking_token.get_full_token_name())

Claim for user

In [ ]:
claim_for_user(user)
advance_blocks(1)

In [ ]:
user = users_init()[0]
claim_on_behalf_from_user(user)
advance_blocks(1)

In [10]:
advance_blocks(1)

Claim on behalf

In [ ]:
user = users_init()[0]
permissions_hub_contract = deploy_permissions_hub()
metastaking_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)

claim_on_behalf_from_user(user)
advance_blocks(1)

Claim compare

In [ ]:
user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
c12 = staking_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_STAKING_STATS_1, u12, "Initial staking user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.STAKING_CONTRACT_STATS_1, c12, "Initial staking contract stats")

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "Initial claim operations")

tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Second claim operations")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
c22 = staking_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_STAKING_STATS_2, u22, "Ending staking user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.STAKING_CONTRACT_STATS_2, c22, "Ending staking contract stats")
collector.add(DictType.TOKEN_ATTRS_1, tk_attrs_1, "Ending token attributes")

# ------------------------------------------------------------------------------------------------
input("Restart Chain Simulator then press Enter to continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
staking_upgrade()  # eats 2 blocks
consumed_blocks += 2
metastaking_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
c12 = staking_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_STAKING_STATS_1, u12, "Initial staking user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.STAKING_CONTRACT_STATS_1, c12, "Initial staking contract stats")

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "Initial claim operations")

tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Second claim operations")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = staking_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
c22 = staking_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_STAKING_STATS_2, u22, "Ending staking user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.STAKING_CONTRACT_STATS_2, c22, "Ending staking contract stats")
collector.add(DictType.TOKEN_ATTRS_1, tk_attrs_1, "Ending token attributes")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim on Behalf compare

In [ ]:
user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = claim_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "First claim ops")

tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Second claim ops")
collect_ending_test_data(collector, user, farm_contract, staking_contract)

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
token = ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_3, "First claim ops")

contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, 
                                                               [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)
collector.add(DictType.OP2, claim_ops_4, "Second claim ops")

collect_ending_test_data(collector, user, farm_contract, staking_contract, dummy_proxy_contract)

close_test(chain_sim, collector)

Partial claim on behalf compare w. enter consolidation

In [ ]:
user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
first_claim_balance = user_tk_balance // 2
remaining_balance = user_tk_balance - first_claim_balance

# first claim of half of the balance
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, first_claim_balance)]
tx_hash = metastaking_contract.claim_rewards_metastaking(context.network_provider.proxy, user, [tokens])
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "First claim ops")

# second claim of the remaining half of the balance
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, remaining_balance)]
tx_hash = metastaking_contract.claim_rewards_metastaking(context.network_provider.proxy, user, [tokens])
advance_blocks(5)
sleep(2)

# enter consolidation
tx_hash = enter_farm_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Consolidated enter ops")
collect_ending_test_data(collector, user, farm_contract, staking_contract)

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
first_claim_balance = user_tk_balance // 2
remaining_balance = user_tk_balance - first_claim_balance

# first claim of half of the balance
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, first_claim_balance)]
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_3, "First claim ops")

# second claim of the remaining half of the balance
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, remaining_balance)]
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(4)
sleep(2)

# transfer back one of the positions to the user then do a hybrid transfer for entering consolidation
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
_ = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, user.address.bech32(), "transfer"])
advance_blocks(1)
sleep(2)

# enter consolidation
user_farming_nonce, user_farming_balance = get_farming_position_for_account(user.address.bech32())
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tokens = [ESDTToken(metastaking_contract.farm_token, user_farming_nonce, user_farming_balance),
          ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)
          ]
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, 
                                                               [tokens, 0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, metastaking_contract.address, "stakeFarmOnBehalf", user.address.bech32()])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)
collector.add(DictType.OP2, claim_ops_4, "Second claim ops")

collect_ending_test_data(collector, user, farm_contract, staking_contract, dummy_proxy_contract)

close_test(chain_sim, collector)

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, ESDTToken

chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
initial_blocks = 10
advance_blocks(initial_blocks)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

user.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

# claim half of owned farm tokens
first_claim_balance = farm_tk_balance // 2
event = ClaimRewardsFarmEvent(first_claim_balance, farm_tk_nonce, '')
tx_hash = farm_contract.claimRewards(context.network_provider, user, event)
advance_blocks(1)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

# claim the rest of the farm tokens
second_claim_balance = farm_tk_balance - first_claim_balance
event = ClaimRewardsFarmEvent(second_claim_balance, farm_tk_nonce, '')
tx_hash = farm_contract.claimRewards(context.network_provider, user, event)
advance_blocks(4)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)

# claim all tokens again - should lead to compounding and rewards given only for first position
tokens_in_account = get_all_token_nonces_details_for_account(metastaking_contract.metastake_token, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {metastaking_contract.metastake_token} in account')
token_payments = []
for token in tokens_in_account:
    token_payments.append(ESDTToken(metastaking_contract.metastake_token, token['nonce'], int(token['balance'])))
gas_limit = 50000000
function = "claimRewards"
sc_args = [
    token_payments
]
tx_hash = multi_esdt_endpoint_call(function, context.network_provider.proxy, gas_limit, user,
                                Address(farm_contract.address), function, sc_args)
advance_blocks(1)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

# claim final position
tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_bf = context.network_provider.get_tx_operations(tx_hash, True)

input("Restarting Chain Simulator. Continue...")
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()

setup_chain_sim()
user = users_init()[2]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2
permissions_hub_contract = deploy_permissions_hub() # eats 1 block
consumed_blocks += 1

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)
consumed_blocks += 1

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block
consumed_blocks += 1

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)
consumed_blocks += 1

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())

# claim half of owned farm tokens
first_claim_balance = user_tk_balance // 2
token = ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, first_claim_balance)
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash, True)

# claim the rest of the farm tokens
second_claim_balance = user_tk_balance - first_claim_balance
token = ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, second_claim_balance)
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_5 = context.network_provider.get_tx_operations(tx_hash, True)

# send one of the positions back to the user (limitation of the dummy proxy contract)
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, user.address, ""])
advance_blocks(3)
sleep(2)

# claim all tokens again via hybrid transfer - should lead to compounding and rewards given only for first position
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
token = ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, [[token], 0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, 
                                                                                                        farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_6 = context.network_provider.get_tx_operations(tx_hash)

u5 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u6 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c4 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_2 = user_farm_token_stats(Account(address=dummy_proxy_contract.address))

# claim final position
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, farm_contract.address, "claimRewardsOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_af = context.network_provider.get_tx_operations(tx_hash)

print(check_equal_dicts(u1, u3))
print(check_equal_dicts(u2, u5))
print(check_equal_dicts(u4, u6))
print(check_equal_dicts(c1, c3))
print(check_equal_dicts(c2, c4))
print("-------------------")
print(tk_attrs_1)
print(tk_attrs_2)
print("-------------------")
print(claim_ops_1)
print(claim_ops_4)
print("-------------------")
print(claim_ops_2)
print(claim_ops_5)
print("-------------------")
print(claim_ops_3)
print(claim_ops_6)
print("-------------------")
print(claim_ops_bf)
print(claim_ops_af)

input("Closing Chain Simulator. Continue...")

stop_chain_sim_stack(chain_sim)

Enter farm compare

In [ ]:
user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collect_ending_test_data(collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_1, "Initial enter ops")

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collect_ending_test_data(collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_3, "Ending enter ops")

close_test(chain_sim, collector)

Enter farm no consolidation compare

In [ ]:
user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_no_consolidation_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collect_ending_test_data(collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_1, "Initial enter ops")
################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_no_consolidation_for_user(user)
advance_blocks(5)

sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collect_ending_test_data(collector, user, farm_contract, staking_contract)
collector.add(DictType.OP1, claim_ops_3, "Ending enter ops")

close_test(chain_sim, collector)

Enter farm on behalf compare

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, ESDTToken

user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "First claim ops")

tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Second claim ops")
collect_ending_test_data(collector, user, farm_contract, staking_contract)

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
farming_token_nonce, farming_tk_balance = get_farming_position_for_account(user.address.bech32())
tokens = [ESDTToken(metastaking_contract.farm_token, farming_token_nonce, farming_tk_balance),
          ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)]
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, metastaking_contract.address, "stakeFarmOnBehalf", user.address.bech32()])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_3, "First claim ops")

contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, 
                                                               [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)
collector.add(DictType.OP2, claim_ops_4, "Second claim ops")

collect_ending_test_data(collector, user, farm_contract, staking_contract, dummy_proxy_contract)

close_test(chain_sim, collector)

Enter on behalf without compounding

In [ ]:
from utils.utils_tx import multi_esdt_endpoint_call, ESDTToken

user_index = 0
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_1, "First claim ops")

tx_hash = claim_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Second claim ops")
collect_ending_test_data(collector, user, farm_contract, staking_contract)

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

farming_token_nonce, farming_tk_balance = get_farming_position_for_account(user.address.bech32())
tokens = [ESDTToken(metastaking_contract.farm_token, farming_token_nonce, farming_tk_balance)]
tx_hash = dummy_proxy_contract.call_transfer_endpoint(user, context.network_provider.proxy, [tokens, 0, metastaking_contract.address, "stakeFarmOnBehalf", user.address.bech32()])
advance_blocks(5)
sleep(2)
claim_ops_3 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_ops_3, "First claim ops")

contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)
tx_hash = dummy_proxy_contract.call_internal_transfer_endpoint(user, context.network_provider.proxy, 
                                                               [0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, metastaking_contract.address, "claimDualYieldOnBehalf"])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)
collector.add(DictType.OP2, claim_ops_4, "Second claim ops")

collect_ending_test_data(collector, user, farm_contract, staking_contract, dummy_proxy_contract)

close_test(chain_sim, collector)

Enter farm on behalf mixed

In [ ]:
from utils.utils_tx import multi_esdt_transfer

user_index = 0
user2_index = 1
initial_blocks = 15

chain_sim, users, collector = init_pre_update_test(initial_blocks)
user = users[user_index]
user2 = users[user2_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

u1 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
su1 = staking_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
collector.add(DictType.USER_2_FARM_STATS_1, u1, "User 2 farm stats 1")
collector.add(DictType.USER_2_STAKING_STATS_1, su1, "User 2 staking stats 1")

# send user's position away
user_tk_nonce, user_tk_balance = get_position_for_account(user.address.bech32())
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)]
multi_esdt_transfer(context.network_provider.proxy, 10000000, user, context.deployer_account.address, tokens)

# send user2's position to user and use this position to enter farm
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)]
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, user.address, tokens)
advance_blocks(1)
sleep(2)

# enter consolidation
tx_hash = enter_farm_for_user(user)
advance_blocks(1)
sleep(2)
claim_ops_2 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_ops_2, "Consolidated enter ops")
collect_ending_test_data(collector, user, farm_contract, staking_contract)

u1 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
su1 = staking_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
collector.add(DictType.USER_2_FARM_STATS_2, u1, "User 2 farm stats 2")
collector.add(DictType.USER_2_STAKING_STATS_2, su1, "User 2 staking stats 2")

################################################################################
chain_sim, users, permissions_hub_contract, dummy_proxy_contract = init_post_upgrade_test(initial_blocks, chain_sim, collector)
user = users[user_index]
user2 = users[user2_index]

collect_initial_test_data(collector, user, farm_contract, staking_contract)

u1 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
su1 = staking_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
collector.add(DictType.USER_2_FARM_STATS_1, u1, "User 2 farm stats 1")
collector.add(DictType.USER_2_STAKING_STATS_1, su1, "User 2 staking stats 1")

# send user2's position to dummy proxy
user_tk_nonce, user_tk_balance = get_position_for_account(user2.address.bech32())
tokens = [ESDTToken(metastaking_contract.metastake_token, user_tk_nonce, user_tk_balance)]
multi_esdt_transfer(context.network_provider.proxy, 10000000, user2, WrapperAddress(dummy_proxy_contract.address), tokens)
advance_blocks(1)
sleep(2)

# enter consolidation
user_farming_nonce, user_farming_balance = get_farming_position_for_account(user.address.bech32())
contract_tk_nonce, contract_tk_balance = get_position_for_account(dummy_proxy_contract.address)

tokens = [ESDTToken(metastaking_contract.farm_token, user_farming_nonce, user_farming_balance)]
tx_hash = dummy_proxy_contract.call_hybrid_transfer_endpoint(user, context.network_provider.proxy, 
                                                               [tokens, 0, metastaking_contract.metastake_token, contract_tk_nonce, contract_tk_balance, metastaking_contract.address, "stakeFarmOnBehalf", user.address.bech32()])
advance_blocks(1)
sleep(2)
claim_ops_4 = context.network_provider.get_tx_operations(tx_hash)
collector.add(DictType.OP2, claim_ops_4, "Consolidated enter ops")

collect_ending_test_data(collector, user, farm_contract, staking_contract, dummy_proxy_contract)

u1 = farm_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
su1 = staking_contract.get_all_user_boosted_stats(user2.address.bech32(), context.network_provider.proxy)
collector.add(DictType.USER_2_FARM_STATS_2, u1, "User 2 farm stats 2")
collector.add(DictType.USER_2_STAKING_STATS_2, su1, "User 2 staking stats 2")

close_test(chain_sim, collector)

Freestyle

In [ ]:
user = users_init()[0]
advance_blocks(3)

u1 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c1 = farm_contract.get_all_stats(context.network_provider.proxy)

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)
claim_ops_1 = context.network_provider.get_tx_operations(tx_hash, True)

u2 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c2 = farm_contract.get_all_stats(context.network_provider.proxy)
tk_attrs_1 = user_farm_token_stats(user)

In [ ]:
farm_upgrade()  # eats 2 blocks

permissions_hub_contract = deploy_permissions_hub() # eats 1 block

farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)
advance_blocks(1)

dummy_proxy_contract = deploy_dummy_proxy_contract()  # eats 1 block

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)

u3 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u4 = farm_contract.get_all_user_boosted_stats(dummy_proxy_contract.address, context.network_provider.proxy)
c3 = farm_contract.get_all_stats(context.network_provider.proxy)

In [ ]:
permissions_hub_contract.add_to_blacklist(context.deployer_account, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)

In [ ]:
permissions_hub_contract.remove_from_blacklist(context.deployer_account, context.network_provider.proxy, [dummy_proxy_contract.address])
advance_blocks(1)


In [ ]:
tx_hash = claim_on_behalf_from_user(user)

In [61]:
advance_blocks(1)